---
# **LAB 6 - Global memory (GMEM)**
---

# ▶️ CUDA setup

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

## [GPU Compute Capability](https://developer.nvidia.com/cuda-gpus)

## NVCC Plugin for Jupyter notebook

*Usage*:


*   Load Extension `%load_ext nvcc_plugin`
*   Mark a cell to be treated as cuda cell
`%%cuda --name example.cu --compile false`

**NOTE**: The cell must contain either code or comments to be run successfully. It accepts 2 arguments. `-n | --name` - which is the name of either CUDA source or Header. The name parameter must have extension `.cu` or `.h`. Second argument -c | --compile; default value is false. The argument is a flag to specify if the cell will be compiled and run right away or not. It might be usefull if you're playing in the main function

*  We are ready to run CUDA C/C++ code right in your Notebook. For this we need explicitly say to the interpreter, that we want to use the extension by adding `%%cu` at the beginning of each cell with CUDA code. 




In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

Clone GPUcomputing site on github...

In [ ]:
!git clone https://github.com/giulianogrossi/GPUcomputing.git

# ▶️ VS Code on Colab

In [ ]:
#@title Colab-ssh tunnel
#@markdown Execute this cell to open the ssh tunnel. Check [colab-ssh documentation](https://github.com/WassimBenzarti/colab-ssh) for more details.

# Install colab_ssh on google colab
!pip install colab_ssh --upgrade

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
ssh_tunnel_password = "gpu" #@param {type: "string"}
launch_ssh_cloudflared(password=ssh_tunnel_password)

# Optional: if you want to clone a Github or Gitlab repository
repository_url="https://github.com/giulianogrossi/GPUcomputing" #@param {type: "string"}
init_git_cloudflared(repository_url)

# ▶️ DeviceQuery

In [ ]:
# DeviceQuery dell'attuale device (su Colab!)
!nvcc /content/GPUcomputing/utils/deviceQuery.cu -o deviceQuery
!./deviceQuery

# ✅ Static and pinned memory

**Pinned memory**

An example of using CUDA's memory copy API to transfer data to and from the device. In this case, `cudaMalloc` is used to allocate memory on the GPU and `cudaMemcpy` is used to transfer the contents of host memory to an array allocated using `cudaMalloc`. Host memory is allocated using `cudaMallocHost` to create a page-locked host array.

In [ ]:
%%cuda --name pinMemTransfer.cu

#include "../GPUcomputing/utils/common.h"
#include <cuda_runtime.h>
#include <stdio.h>

int main(int argc, char **argv) {
  //# set up device
  int dev = 0;
  CHECK(cudaSetDevice(dev));

  //# memory size
  unsigned long isize = 1 << 30;
  unsigned long nbytes = isize * sizeof(float);

  //# get device information
  cudaDeviceProp deviceProp;
  CHECK(cudaGetDeviceProperties(&deviceProp, dev));

  //# check if device support pinned memory
  if (!deviceProp.canMapHostMemory) {
      printf("Device %d does not support mapping CPU host memory!\n", dev);
      CHECK(cudaDeviceReset());
      exit(EXIT_SUCCESS);
  }

  printf("device %d: %s memory size = %lu byte (%5.2f MB) and canMapHostMemory = %d\n", dev,
          deviceProp.name, isize, nbytes / (1024.0f * 1024.0f),
          deviceProp.canMapHostMemory);

  float *h_a;
  //# allocate the host memory   
  //h_a = (float *)malloc(nbytes);

  //# allocate pinned host memory
  CHECK(cudaMallocHost ((float **)&h_a, nbytes));

  //# allocate device memory
  float *p_a;
  CHECK(cudaMalloc((float **)&p_a, nbytes));

  for (int i = 0; i < isize; i++) 
    h_a[i] = 100.10f;

  //# transfer data from the host to the device
  CHECK(cudaMemcpy(p_a, h_a, nbytes, cudaMemcpyHostToDevice));

  //# transfer data from the device to the host
  CHECK(cudaMemcpy(h_a, p_a, nbytes, cudaMemcpyDeviceToHost));

  //# free memory
  CHECK(cudaFree(p_a));
  CHECK(cudaFreeHost(h_a));

  // reset device
  CHECK(cudaDeviceReset());
  return EXIT_SUCCESS;
}


In [ ]:
!nvcc -O3 -arch=sm_75 src/pinMemTransfer.cu -o pinMemTransfer
!nvprof ./pinMemTransfer

# ✅ Unified memory


In [ ]:
%%cuda --name sumMatrix.cu

#include <stdio.h>
#include "../GPUcomputing/utils/common.h"

void initialData(float *ip, const int size) {
  int i;

  for (i = 0; i < size; i++)
    ip[i] = (float)( rand() & 0xFF ) / 10.0f;
  return;
}

void sumMatrixOnHost(float *A, float *B, float *C, const int nx, const int ny) {
  float *ia = A;
  float *ib = B;
  float *ic = C;

  for (int iy = 0; iy < ny; iy++) {
    for (int ix = 0; ix < nx; ix++)
      ic[ix] = ia[ix] + ib[ix];

    ia += nx;
    ib += nx;
    ic += nx;
  }
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  bool match = 1;

  for (int i = 0; i < N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      match = 0;
      printf("host %f gpu %f\n", hostRef[i], gpuRef[i]);
      break;
    }
  }

  if (!match)
    printf("Arrays do not match.\n\n");
}

//# matrix sum with grid 2D block 2D
__global__ void sumMatrixGPU(float *MatA, float *MatB, float *MatC, int nx, int ny) {
  unsigned int ix = threadIdx.x + blockIdx.x * blockDim.x;
  unsigned int iy = threadIdx.y + blockIdx.y * blockDim.y;
  unsigned int idx = iy * nx + ix;

  if (ix < nx && iy < ny)
    MatC[idx] = MatA[idx] + MatB[idx];
}

//# MAIN
int main(int argc, char **argv) {
    printf("%s Starting ", argv[0]);

    //# set up device
    int dev = 0;
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("using Device %d: %s\n", dev, deviceProp.name);
    CHECK(cudaSetDevice(dev));

    //# set up data size of matrix
    int nx, ny;
    int ishift = 12;
    if  (argc > 1) ishift = atoi(argv[1]);
    nx = ny = 1 << ishift;

    int nxy = nx * ny;
    int nBytes = nxy * sizeof(float);
    printf("Matrix size: nx %d ny %d\n", nx, ny);

    //# malloc host memory
    float *h_A, *h_B, *hostRef, *gpuRef;
    h_A = (float *)malloc(nBytes);
    h_B = (float *)malloc(nBytes);
    hostRef = (float *)malloc(nBytes);
    gpuRef = (float *)malloc(nBytes);

    // initialize data at host side
    double iStart = seconds();
    initialData(h_A, nxy);
    initialData(h_B, nxy);
    double iElaps = seconds() - iStart;

    printf("initialization: \t %f sec\n", iElaps);

    memset(hostRef, 0, nBytes);
    memset(gpuRef, 0, nBytes);

    //# add matrix at host side for result checks
    iStart = seconds();
    sumMatrixOnHost(h_A, h_B, hostRef, nx, ny);
    iElaps = seconds() - iStart;
    printf("sumMatrix on host:\t %f sec\n", iElaps);

    //# malloc device global memory
    float *d_MatA, *d_MatB, *d_MatC;
    CHECK(cudaMalloc((void **)&d_MatA, nBytes));
    CHECK(cudaMalloc((void **)&d_MatB, nBytes));
    CHECK(cudaMalloc((void **)&d_MatC, nBytes));

    //# transfer data from host to device
    CHECK(cudaMemcpy(d_MatA, h_A, nBytes, cudaMemcpyHostToDevice));
    CHECK(cudaMemcpy(d_MatB, h_B, nBytes, cudaMemcpyHostToDevice));
    
    //# invoke kernel at host side
    int dimx = 32;
    int dimy = 32;
    dim3 block(dimx, dimy);
    dim3 grid((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

    iStart =  seconds();
    sumMatrixGPU<<<grid, block>>>(d_MatA, d_MatB, d_MatC, nx, ny);
    CHECK(cudaDeviceSynchronize());
    iElaps = seconds() - iStart;
    printf("sumMatrix on gpu :\t %f sec <<<(%d,%d), (%d,%d)>>> \n", iElaps, grid.x, grid.y, block.x, block.y);
    CHECK(cudaMemcpy(gpuRef, d_MatC, nBytes, cudaMemcpyDeviceToHost));
    //# check kernel error
    CHECK(cudaGetLastError());
    //# check device results
    checkResult(hostRef, gpuRef, nxy);

    // free device global memory
    CHECK(cudaFree(d_MatA));
    CHECK(cudaFree(d_MatB));
    CHECK(cudaFree(d_MatC));

    // free host memory
    free(h_A);
    free(h_B);
    free(hostRef);
    free(gpuRef);

    // reset device
    CHECK(cudaDeviceReset());

    return (0);
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_70 src/sumMatrix.cu  -o sumMatrix
!./sumMatrix 14

In [ ]:
# profilazione

!nvprof ./sumMatrix 14

# 🔴 TODO

1. Definire la UMEM per ogni matrice
2. Effettuare la somma invocando il kernel
3. Analizzare i tempi e prestazioni con nvprof


In [ ]:
%%cuda --name sumMatrixUni.cu

#include "../GPUcomputing/utils/common.h"

void initialData(float *ip, const int size) {
  int i;

  for (i = 0; i < size; i++)
    ip[i] = (float)( rand() & 0xFF ) / 10.0f;
  return;
}

void sumMatrixOnHost(float *A, float *B, float *C, const int nx, const int ny) {
  float *ia = A;
  float *ib = B;
  float *ic = C;

  for (int iy = 0; iy < ny; iy++) {
    for (int ix = 0; ix < nx; ix++)
      ic[ix] = ia[ix] + ib[ix];

    ia += nx;
    ib += nx;
    ic += nx;
  }
  return;
}

void checkResult(float *hostRef, float *gpuRef, const int N) {
  double epsilon = 1.0E-8;
  bool match = 1;

  for (int i = 0; i < N; i++) {
    if (abs(hostRef[i] - gpuRef[i]) > epsilon) {
      match = 0;
      printf("host %f gpu %f\n", hostRef[i], gpuRef[i]);
      break;
    }
  }

  if (!match)
    printf("Arrays do not match.\n\n");
}

//# grid 2D block 2D
__global__ void sumMatrixGPU(float *MatA, float *MatB, float *MatC, int nx, int ny) {
  unsigned int ix = threadIdx.x + blockIdx.x * blockDim.x;
  unsigned int iy = threadIdx.y + blockIdx.y * blockDim.y;
  unsigned int idx = iy * nx + ix;

  if (ix < nx && iy < ny)
    MatC[idx] = MatA[idx] + MatB[idx];
}

//# MAIN
int main(int argc, char **argv) {
  printf("%s Starting ", argv[0]);

  // set up data size of matrix
  int nx, ny;
  int ishift = 14;
  if  (argc > 1) ishift = atoi(argv[1]);
  nx = ny = 1 << ishift;

  int nxy = nx * ny;
  int nBytes = nxy * sizeof(float);
  printf("Matrix size: nx %d ny %d\n", nx, ny);

  //# malloc unified host memory
  float *A, *B, *gpuRef;
  CHECK(cudaMallocManaged((void **)&A, nBytes));
  CHECK(cudaMallocManaged((void **)&B, nBytes));
  CHECK(cudaMallocManaged((void **)&gpuRef,  nBytes);  );

  //# invoke kernel at host side
  int dimx = 32;
  int dimy = 32;
  dim3 block(dimx, dimy);
  dim3 grid((nx + block.x - 1) / block.x, (ny + block.y - 1) / block.y);

  //# after warm-up, time with unified memory
  double iStart = seconds();
  sumMatrixGPU<<<grid, block>>>(A, B, gpuRef, nx, ny);
  CHECK(cudaDeviceSynchronize());
  double iElaps = seconds() - iStart;
  printf("sumMatrix on gpu :\t %f sec <<<(%d,%d), (%d,%d)>>> \n", iElaps, grid.x, grid.y, block.x, block.y);
  //# check kernel error
  CHECK(cudaGetLastError());

  //# free device global memory
  CHECK(cudaFree(A));
  CHECK(cudaFree(B));
  CHECK(cudaFree(gpuRef));

  // reset device
  CHECK(cudaDeviceReset());

  return (0);
}


In [ ]:
# Compilazione ed esecuzione

!nvcc -arch=sm_70 src/sumMatrixUni.cu  -o sumMatrixUni
!./sumMatrixUni 14

In [ ]:
# profilazione

!nvprof ./sumMatrixUni

# ✅ SoA vs AoS structs

In [ ]:
%%cuda --name SoA.cu

#include <stdint.h>
#include "../GPUcomputing/utils/common.h"

#define N 1<<25
#define blocksize 1<<7

struct SoA {
	uint8_t r[N];
	uint8_t g[N];
	uint8_t b[N];
};


void initialize(SoA*, int);
void checkResult(SoA*, SoA*, int);

/*
 * Riscala l'immagine al valore massimo [max] fissato
 */
__global__ void rescaleImg(SoA *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		SoA *tmp = img;
		r = max * (float)tmp->r[i]/255.0f;
		img->r[i] = (uint8_t)r;
		g = max * (float)tmp->g[i]/255.0f;
		img->g[i] = (uint8_t)g;
		b = max * (float)tmp->b[i]/255.0f;
		img->b[i] = (uint8_t)b;
	}
}

/*
 * cancella un piano dell'immagine [plane = 'r' o 'g' o 'b'] fissato
 */
__global__ void deletePlane(SoA *img, const char plane, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		switch (plane) {
		case 'r':
			img->r[i] = 0;
			break;
		case 'g':
			img->g[i] = 0;
			break;
		case 'b':
			img->b[i] = 0;
			break;
		}
	}
}

/*
 * setup device
 */
__global__ void warmup(SoA *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		SoA *tmp = img;
		r = max * (float)tmp->r[i]/255.0f;
		img->r[i] = (uint8_t)r;
		g = max * (float)tmp->g[i]/255.0f;
		img->g[i] = (uint8_t)g;
		b = max * (float)tmp->b[i]/255.0f;
		img->b[i] = (uint8_t)b;
	}
}

/*
 * Legge da stdin quale kernel eseguire: 0 per rescaleImg, 1 per deletePlane
 */
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s test SoA at ", argv[0]);
	printf("device %d: %s \n", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	// scelta del kernel da eseguire
	int kernel = 0;
	if (argc > 1) kernel = atoi(argv[1]);

	// allocate host memory
	size_t nBytes = sizeof(SoA);
	SoA *img = (SoA *)malloc(nBytes);
	SoA *new_img = (SoA *)malloc(nBytes);

	// initialize host array
	initialize(img, N);

	// allocate device memory
	int n_elem = N;
	SoA *d_img;
	CHECK(cudaMalloc((void**)&d_img, nBytes));

	// copy data from host to device
	CHECK(cudaMemcpy(d_img, img, nBytes, cudaMemcpyHostToDevice));

	// definizione max
	int max = 128;
	if (argc > 2) max = atoi(argv[2]);

	// configurazione per esecuzione
	dim3 block (blocksize, 1);
	dim3 grid  ((n_elem + block.x - 1) / block.x, 1);

	// kernel 1: warmup
	double iStart = seconds();
	warmup<<<1, 32>>>(d_img, max, 32);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;
	printf("warmup<<< 1, 32 >>> elapsed %f sec\n",iElaps);
	CHECK(cudaGetLastError());

	// kernel 2 rescaleImg o deletePlane
	iStart = seconds();
	if (kernel == 0) {
		rescaleImg<<<grid, block>>>(d_img, max, n_elem);
	}
	else {
		deletePlane<<<grid, block>>>(d_img, 'r', n_elem);
	}

	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	CHECK(cudaMemcpy(new_img, d_img, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaGetLastError());

	if (kernel == 0) {
		printf("rescaleImg <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}
	else {
		printf("deletePlane <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}

	//checkResult(img, new_img, n_elem);

	// free memories both host and device
	CHECK(cudaFree(d_img));
	free(img);
	free(new_img);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialize(SoA *img,  int size) {
	for (int i = 0; i < size; i++) {
		img->r[i] = rand() % 256;
		img->g[i] = rand() % 256;
		img->b[i] = rand() % 256;
	}
	return;
}

void checkResult(SoA *img, SoA *new_img, int n_elem) {
	for (int i = 0; i < n_elem; i+=1000)
		printf("img[%d] = (%d,%d,%d) -- new_img[%d] = (%d,%d,%d)\n",
				i,img->r[i],img->g[i],img->b[i],i,new_img->r[i],new_img->g[i],new_img->b[i]);
	return;
}


void transposeHost(float *out, float *in, const int nx, const int ny) {
	for (int iy = 0; iy < ny; ++iy) {
		for (int ix = 0; ix < nx; ++ix) {
			out[ix * ny + iy] = in[iy * nx + ix];
		}
	}
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70  src/SoA.cu -o SoA
!./SoA 1

# 🔴 TODO

In [ ]:
%%cuda --name AoS.cu

#include <stdint.h>
#include "../GPUcomputing/utils/common.h"

#define N 1<<25
#define blocksize 128

struct AoS {
	uint8_t r;
	uint8_t g;
	uint8_t b;
};

void initialize(AoS *, int);
void checkResult(AoS *, AoS *, int);

/*
 * Riscala l'immagine al valore massimo [max] fissato
 */
__global__ void rescaleImg(AoS *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		AoS tmp = img[i];
		r = max * (float)tmp.r/255.0f;
		tmp.r = (uint8_t)r;
		g = max * (float)tmp.g/255.0f;
		tmp.g = (uint8_t)g;
		b = max * (float)tmp.b/255.0f;
		tmp.b = (uint8_t)b;
		img[i] = tmp;
	}
}

/*
 * cancella un piano dell'immagine [plane = 'r' o 'g' o 'b'] fissato
 */
__global__ void deletePlane(AoS *img, const char plane, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		switch (plane) {
		case 'r':
			img[i].r = 0;
			break;
		case 'g':
			img[i].g = 0;
			break;
		case 'b':
			img[i].b = 0;
			break;
		}
	}
}

/*
 * setup device
 */
__global__ void warmup(AoS *img, const int max, const int n) {
	unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n) {
		float r,g,b;
		AoS tmp = img[i];
		r = max * (float)tmp.r/255.0f;
		tmp.r = (uint8_t)r;
		g = max * (float)tmp.g/255.0f;
		tmp.g = (uint8_t)g;
		b = max * (float)tmp.b/255.0f;
		tmp.b = (uint8_t)b;
		img[i] = tmp;
	}
}

/*
 * Legge da stdin quale kernel eseguire: 0 per rescaleImg, 1 per deletePlane
 */
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s test AoS at ", argv[0]);
	printf("device %d: %s \n", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	// scelta del kernel da eseguire
	int kernel = 0;
	if (argc > 1) kernel = atoi(argv[1]);

	// allocate host memory
	int n_elem = N;
	size_t nBytes = n_elem * sizeof(AoS);
	AoS *img = (AoS *)malloc(nBytes);
	AoS *new_img = (AoS *)malloc(nBytes);

	// initialize host array
	initialize(img, N);

	// allocate device memory
	AoS *d_img;
	CHECK(cudaMalloc((void**)&d_img, nBytes));

	// copy data from host to device
	CHECK(cudaMemcpy(d_img, img, nBytes, cudaMemcpyHostToDevice));

	// definizione max
	int max = 128;
	if (argc > 2) max = atoi(argv[2]);

	// configurazione per esecuzione
	dim3 block (blocksize, 1);
	dim3 grid  ((n_elem + block.x - 1) / block.x, 1);

	// kernel 1: warmup
	double iStart = seconds();
	warmup<<<1, 32>>>(d_img, max, 32);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;
	printf("warmup<<< 1, 32 >>> elapsed %f sec\n",iElaps);
	CHECK(cudaGetLastError());

	// kernel 2 rescaleImg o deletePlane
	iStart = seconds();
	if (kernel == 0) {
		rescaleImg<<<grid, block>>>(d_img, max, n_elem);
	}
	else {
		deletePlane<<<grid, block>>>(d_img, 'r', n_elem);
	}
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;
	CHECK(cudaMemcpy(new_img, d_img, nBytes, cudaMemcpyDeviceToHost));
	CHECK(cudaGetLastError());

	if (kernel == 0) {
		printf("rescaleImg <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}
	else {
		printf("deletePlane <<< %3d, %3d >>> elapsed %f sec\n", grid.x, block.x, iElaps);
	}
	//checkResult(img, new_img, n_elem);

	// free memories both host and device
	CHECK(cudaFree(d_img));
	free(img);
	free(new_img);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialize(AoS *img,  int size) {
	for (int i = 0; i < size; i++) {
		img[i].r = rand() % 256;
		img[i].g = rand() % 256;
		img[i].b = rand() % 256;
	}
	return;
}

void checkResult(AoS *img, AoS *new_img, int n_elem) {
	for (int i = 0; i < n_elem; i+=1000)
		printf("img[%d] = (%d,%d,%d) -- new_img[%d] = (%d,%d,%d)\n",
				i,img[i].r,img[i].g,img[i].b,i,new_img[i].r,new_img[i].g,new_img[i].b);
	return;
}



In [ ]:
# Compilazione ed esecuzione
!nvcc -arch=sm_70 src/AoS.cu -o AoS
!./AoS 1

# ✅ Transpose

# 🔴 TODO

passi per la trasposizione con SMEM:

1. definire la dim della SMEM pari alla dim del blocco
2. Il warp scrive i dati nella shared memory in row-major ordering evitando bank conflict sulle scritture. Ogni warp fa una letture coalescente dei dati in global memory
3. sincronizzare i thread


In [ ]:
%%cuda --name transposeSMEM.cu

#include <stdio.h>
#include "../GPUcomputing/utils/common.h"


// Dimensione del blocco
#define BDIMX 32
#define BDIMY 32

// macro x conversione indici lineari
#define INDEX(rows, cols, stride) (rows * stride + cols)

// prototipi funzioni
void initialData(float*, const int);
void printData(float*, int, int);
void checkResult(float*, float*, int, int);
void transposeHost(float*, float*, const int, const int);

/*
 * Kernel per il calcolo della matrice trasposta usando la shared memory
 */
__global__ void transposeSmem(float *out, float *in, int nrows, int ncols) {
	// static shared memory
	__shared__ float tile[BDIMY][BDIMX];

	// coordinate matrice originale
	//unsigned int row = blockDim.y * blockIdx.y + threadIdx.y;
	//unsigned int col = blockDim.x * blockIdx.x + threadIdx.x;

	unsigned int y = blockDim.y * blockIdx.y + threadIdx.y;
	unsigned int x = blockDim.x * blockIdx.x + threadIdx.x;

	// trasferimento dati dalla global memory alla shared memory
	if (y < nrows && x < ncols)
		tile[threadIdx.y][threadIdx.x] = in[INDEX(y, x, ncols)];

	// thread synchronization
	__syncthreads();

	// offset blocco trasposto
	y = blockIdx.x * blockDim.x + threadIdx.y;
	x = blockIdx.y * blockDim.y + threadIdx.x;

	// controlli invertiti nelle dim riga colonna
	if (y < ncols && x < nrows)
		out[y*nrows + x] = tile[threadIdx.x][threadIdx.y];
}

//# naive: access data in rows
__global__ void copyRow(float *out, float *in, const int nrows,	const int ncols) {
	// matrix coordinate (ix,iy)
	unsigned int row = blockIdx.y * blockDim.y + threadIdx.y;
	unsigned int col = blockIdx.x * blockDim.x + threadIdx.x;

	// transpose with boundary test
	if (row < nrows && col < ncols)
		out[INDEX(col, row, nrows)] = in[INDEX(row, col, ncols)];
}

//# naive: access data in cols
__global__ void copyCol(float *out, float *in, const int nrows,	const int ncols) {
	// matrix coordinate (ix,iy)
	unsigned int row = blockIdx.y * blockDim.y + threadIdx.y;
	unsigned int col = blockIdx.x * blockDim.x + threadIdx.x;

	// transpose with boundary test
	if (row < nrows && col < ncols)
		out[INDEX(row, col, ncols)] = in[INDEX(col, row, nrows)];
}

//# MAIN
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s starting transpose at ", argv[0]);
	printf("device %d: %s ", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	bool iprint = 0;

	// set up array size
	int nrows = 1 << 14;
	int ncols = 1 << 14;

	if (argc > 1)
		iprint = atoi(argv[1]);
	if (argc > 2)
		nrows = atoi(argv[2]);
	if (argc > 3)
		ncols = atoi(argv[3]);

	printf("\nMatrice con nrows = %d ncols = %d\n", nrows, ncols);
	size_t ncells = nrows * ncols;
	size_t nBytes = ncells * sizeof(float);

	// allocate host memory
	float *A_h = (float *) malloc(nBytes);

	// Allocate Unified Memory – accessible from CPU or GPU
	float *A, *AT;
	cudaMallocManaged(&A, nBytes);
	cudaMallocManaged(&AT, nBytes);

	//  initialize host array
	initialData(A, nrows * ncols);
	if (iprint)
		printData(A, nrows, ncols);

	//  transpose at host side
	transposeHost(A_h, A, nrows, ncols);

	
  printf("*** KERNEL: col copy  ***\n");
	// tranpose gmem
  memset(AT, 0, nBytes);
  dim3 block(BDIMX, BDIMY, 1);
	dim3 grid((ncols + block.x - 1) / block.x, (nrows + block.y - 1) / block.y, 1);
	double iStart = seconds();
	copyCol<<<grid, block>>>(AT, A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;

	// check result
	checkResult(A_h, AT, nrows, ncols);

	double ibnd = 2 * ncells * sizeof(float) / 1e9 / iElaps;
	printf("col copy elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n\n", iElaps, grid.x, grid.y, block.x,	block.y, ibnd);

  
  printf("*** KERNEL: row copy  ***\n");
	// tranpose gmem
  memset(AT, 0, nBytes);

	iStart = seconds();
	copyRow<<<grid, block>>>(AT, A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;

	// check result
	checkResult(A_h, AT, nrows, ncols);

	ibnd = 2 * ncells * sizeof(float) / 1e9 / iElaps;
	printf("row copy elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n\n", iElaps, grid.x, grid.y, block.x,	block.y, ibnd);



	printf("*** KERNEL: transposeSmem ***\n");
	// tranpose smem
	memset(AT, 0, nBytes);

	iStart = seconds();
	transposeSmem<<<grid, block>>>(AT, A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	double iElapsSMEM = seconds() - iStart;

	if (iprint)
		printData(AT, ncols, nrows);

	checkResult(A_h, AT, nrows, ncols);
	ibnd = 2 * ncells * sizeof(float) / 1e9 / iElapsSMEM;
	printf("transposeSmem elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n", iElapsSMEM, grid.x, grid.y, block.x,
			block.y, ibnd);

	printf("SPEEDUP = %f\n", iElaps/iElapsSMEM);

	// free host and device memory
	CHECK(cudaFree(A));
	CHECK(cudaFree(AT));
	free(A_h);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialData(float *in, const int size) {
	for (int i = 0; i < size; i++)
		in[i] = i; // (float)(rand()/INT_MAX) * 10.0f;
	return;
}

void printData(float *in, int nrows, int ncols) {
	for (int i = 0; i < nrows; i++) {
		for (int j = 0; j < ncols; j++)
			printf("%3.0f ", in[INDEX(i, j, ncols)]);
		printf("\n");
	}
}

void transposeHost(float *out, float *in, const int nrows, const int ncols) {
	for (int iy = 0; iy < nrows; ++iy)
		for (int ix = 0; ix < ncols; ++ix)
			out[INDEX(ix, iy, nrows)] = in[INDEX(iy, ix, ncols)];
}

void checkResult(float *hostRef, float *gpuRef, int rows, int cols) {
	double epsilon = 1.0E-8;
	bool match = 1;

	for (int i = 0; i < rows; i++) {
		for (int j = 0; j < cols; j++) {
			int index = INDEX(i, j, cols);
			if (abs(hostRef[index] - gpuRef[index]) > epsilon) {
				match = 0;
				printf("different on (%d, %d) (offset=%d) element in "
						"transposed matrix: host %f gpu %f\n", i, j, index,
						hostRef[index], gpuRef[index]);
				break;
			}
		}
		if (!match)
			break;
	}

	if (!match)
		printf("Arrays do not match.\n");
}


In [ ]:
!nvcc -arch=sm_75 src/transposeSMEM.cu -o transposeSMEM
!./transposeSMEM

In [ ]:
%%cuda --name transposeSMEM.cu

#include <stdio.h>
#include "../GPUcomputing/utils/common.h"


// Dimensione del blocco
#define BDIMX 32
#define BDIMY 32

// macro x conversione indici lineari
#define INDEX(rows, cols, stride) (rows * stride + cols)

// prototipi funzioni
void initialData(float*, const int);
void printData(float*, int, int);
void checkResult(float*, float*, int, int);
void transposeHost(float*, float*, const int, const int);

/*
 * Kernel per il calcolo della matrice trasposta usando la shared memory
 */
__global__ void transposeSmem(float *out, float *in, int nrows, int ncols) {
	// static shared memory
	__shared__ float tile[BDIMY][BDIMX];

	// coordinate matrice originale
	//unsigned int row = blockDim.y * blockIdx.y + threadIdx.y;
	//unsigned int col = blockDim.x * blockIdx.x + threadIdx.x;

	unsigned int y = blockDim.y * blockIdx.y + threadIdx.y;
	unsigned int x = blockDim.x * blockIdx.x + threadIdx.x;

	// trasferimento dati dalla global memory alla shared memory
	if (y < nrows && x < ncols)
		tile[threadIdx.y][threadIdx.x] = in[INDEX(y, x, ncols)];

	// thread synchronization
	__syncthreads();

	// offset blocco trasposto
	y = blockIdx.x * blockDim.x + threadIdx.y;
	x = blockIdx.y * blockDim.y + threadIdx.x;

	// controlli invertiti nelle dim riga colonna
	if (y < ncols && x < nrows)
		out[y*nrows + x] = tile[threadIdx.x][threadIdx.y];
}

//# naive: access data in rows
__global__ void copyRow(float *out, float *in, const int nrows,	const int ncols) {
	// matrix coordinate (ix,iy)
	unsigned int row = blockIdx.y * blockDim.y + threadIdx.y;
	unsigned int col = blockIdx.x * blockDim.x + threadIdx.x;

	// transpose with boundary test
	if (row < nrows && col < ncols)
		out[INDEX(col, row, nrows)] = in[INDEX(row, col, ncols)];
}

//# naive: access data in cols
__global__ void copyCol(float *out, float *in, const int nrows,	const int ncols) {
	// matrix coordinate (ix,iy)
	unsigned int row = blockIdx.y * blockDim.y + threadIdx.y;
	unsigned int col = blockIdx.x * blockDim.x + threadIdx.x;

	// transpose with boundary test
	if (row < nrows && col < ncols)
		out[INDEX(row, col, ncols)] = in[INDEX(col, row, nrows)];
}

//# MAIN
int main(int argc, char **argv) {
	// set up device
	int dev = 0;
	cudaDeviceProp deviceProp;
	CHECK(cudaGetDeviceProperties(&deviceProp, dev));
	printf("%s starting transpose at ", argv[0]);
	printf("device %d: %s ", dev, deviceProp.name);
	CHECK(cudaSetDevice(dev));

	bool iprint = 0;

	// set up array size
	int nrows = 1 << 14;
	int ncols = 1 << 14;

	if (argc > 1)
		iprint = atoi(argv[1]);
	if (argc > 2)
		nrows = atoi(argv[2]);
	if (argc > 3)
		ncols = atoi(argv[3]);

	printf("\nMatrice con nrows = %d ncols = %d\n", nrows, ncols);
	size_t ncells = nrows * ncols;
	size_t nBytes = ncells * sizeof(float);

	// allocate host memory
	float *A_h = (float *) malloc(nBytes);
  float *B_h = (float *) malloc(nBytes);
  float *AT_h = (float *) malloc(nBytes);

	// Allocate Unified Memory – accessible from CPU or GPU
	float *d_A, *d_AT;
	CHECK(cudaMalloc((void** )&d_A, nBytes));
  CHECK(cudaMalloc((void** )&d_AT, nBytes));

	//  initialize host array
	initialData(A_h, nrows * ncols);
	if (iprint)
		printData(A_h, nrows, ncols);
  
	//  transpose at host side
	transposeHost(A_h, B_h, nrows, ncols);

	
  printf("*** KERNEL: col copy  ***\n");
	// tranpose gmem
  CHECK(cudaMemcpy(d_A, A_h, nBytes, cudaMemcpyHostToDevice));
  dim3 block(BDIMX, BDIMY, 1);
	dim3 grid((ncols + block.x - 1) / block.x, (nrows + block.y - 1) / block.y, 1);
	
  double iStart = seconds();
	copyCol<<<grid, block>>>(d_AT, d_A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	double iElaps = seconds() - iStart;

	// check result
	CHECK(cudaMemcpy(AT_h, d_AT, nBytes, cudaMemcpyDeviceToHost));
	checkResult(A_h, B_h, nrows, ncols);

	double ibnd = 2 * ncells * sizeof(float) / 1e9 / iElaps;
	printf("col copy elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n\n", iElaps, grid.x, grid.y, block.x,	block.y, ibnd);

  
  printf("*** KERNEL: row copy  ***\n");
	// tranpose gmem

	iStart = seconds();
	copyRow<<<grid, block>>>(d_AT, d_A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	iElaps = seconds() - iStart;

	// check result
  CHECK(cudaMemcpy(AT_h, d_AT, nBytes, cudaMemcpyDeviceToHost));
	checkResult(A_h, B_h, nrows, ncols);

	ibnd = 2 * ncells * sizeof(float) / 1e9 / iElaps;
	printf("row copy elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n\n", iElaps, grid.x, grid.y, block.x,	block.y, ibnd);


	printf("*** KERNEL: transposeSmem ***\n");
	// tranpose smem

	iStart = seconds();
	transposeSmem<<<grid, block>>>(d_AT, d_A, nrows, ncols);
	CHECK(cudaDeviceSynchronize());
	double iElapsSMEM = seconds() - iStart;

	CHECK(cudaMemcpy(AT_h, d_AT, nBytes, cudaMemcpyDeviceToHost));
	checkResult(A_h, B_h, nrows, ncols);
  
	ibnd = 2 * ncells * sizeof(float) / 1e9 / iElapsSMEM;
	printf("transposeSmem elapsed %f sec\n <<< grid (%d,%d) block (%d,%d)>>> "
			"effective bandwidth %f GB\n", iElapsSMEM, grid.x, grid.y, block.x,
			block.y, ibnd);

	printf("SPEEDUP = %f\n", iElaps/iElapsSMEM);

	// free host and device memory
	CHECK(cudaFree(d_A));
	CHECK(cudaFree(d_AT));
	free(A_h);

	// reset device
	CHECK(cudaDeviceReset());
	return EXIT_SUCCESS;
}

void initialData(float *in, const int size) {
	for (int i = 0; i < size; i++)
		in[i] = i; // (float)(rand()/INT_MAX) * 10.0f;
	return;
}

void printData(float *in, int nrows, int ncols) {
	for (int i = 0; i < nrows; i++) {
		for (int j = 0; j < ncols; j++)
			printf("%3.0f ", in[INDEX(i, j, ncols)]);
		printf("\n");
	}
}

void transposeHost(float *out, float *in, const int nrows, const int ncols) {
	for (int iy = 0; iy < nrows; ++iy)
		for (int ix = 0; ix < ncols; ++ix)
			out[INDEX(ix, iy, nrows)] = in[INDEX(iy, ix, ncols)];
}

void checkResult(float *hostRef, float *gpuRef, int rows, int cols) {
	double epsilon = 1.0E-8;
	bool match = 1;

	for (int i = 0; i < rows; i++) {
		for (int j = 0; j < cols; j++) {
			int index = INDEX(i, j, cols);
			if (abs(hostRef[index] - gpuRef[index]) > epsilon) {
				match = 0;
				printf("different on (%d, %d) (offset=%d) element in "
						"transposed matrix: host %f gpu %f\n", i, j, index,
						hostRef[index], gpuRef[index]);
				break;
			}
		}
		if (!match)
			break;
	}

	if (!match)
		printf("Arrays do not match.\n");
}


In [ ]:
!nvcc -arch=sm_75 src/transposeSMEM.cu -o transposeSMEM
!./transposeSMEM